In [107]:
import ETL_functions as etf
import numpy as np
import pandas as pd
import ast

# steam_games.json

In [108]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True,dtype=str)#lo cargamos como str ya que la mayoria de sus columnas requieren 
#este formato ahorrando asi :
# tener que transformar individualmente las columnas que contienen listas 
# cambiar el valor float de 0 y 1 early access a boolean
#a su vez esto transformara todos los nulls a cadenas de texto las cuales deberemos acceder atravez de el valor "None"(string)
#por lo que ya no podremos utilizar las funciones para el manejo de nulos de pandas

In [109]:
df_sg.shape #las dimensiones de el dataframe son de 120445 filas y 13 columnas

(120445, 13)

In [110]:
df_sg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   publisher     120445 non-null  object
 1   genres        120445 non-null  object
 2   app_name      120445 non-null  object
 3   title         120445 non-null  object
 4   url           120445 non-null  object
 5   release_date  120445 non-null  object
 6   tags          120445 non-null  object
 7   reviews_url   120445 non-null  object
 8   specs         120445 non-null  object
 9   price         120445 non-null  object
 10  early_access  120445 non-null  object
 11  id            120445 non-null  object
 12  developer     120445 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


## correccion los problemas en los datos encontrados durante la primer parte de el eda

### eliminacion de duplicados

In [111]:
df_sg.duplicated().value_counts()#tiene 88309 registros duplicados los cuales no nos interesan

True     88309
False    32136
Name: count, dtype: int64

In [112]:
df_sg.drop_duplicates(inplace=True)#dropeo duplicados lo que incluye la mayoria de columnas con valores nulls

In [113]:
df_sg.shape #las dimensiones de el dataframe sin duplicados son de 32136 filas y 13 columnas

(32136, 13)

In [117]:
df_sg.head(5)#(para asegurarnos que ahora que borro todas las columnas repetidas, solo nos dejo una unica fila con solo valores None la que dropearemos)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570,None


In [115]:
df_sg.drop(0,inplace=True)#con esto borramos la unica columna que no poseia datos que quedaba

### eliminacion de nulos 

debido a que la mayor cantidad de nulos fueron removidos durante el drop_duplicated() solo tratare los None de columnas que considere importantes que no tengan None

In [122]:
df_sg.shape#32135

(32126, 13)

In [119]:
mask = ((df_sg["title"] == "None") & (df_sg["app_name"] == "None")) #filtro title y app_name == "None"
df_sg = df_sg[~mask]# ~ sirve para dropear , aca estamos dropeando las filas en que tienen title "None"
#esto solo borro 2 registros pero eran registros que no tenian utilidad ya que una app que no tiene ningun nombre no se puede recomendar

In [121]:
mask = (df_sg["genres"] == "None") & (df_sg["tags"] == "None") & (df_sg["specs"] == "None")# filtro genres,tags,specs == "None"
df_sg = df_sg[~mask]# borramos las columnas que cumplan con el filtro
#esto borro 7 registros que no tenian ningun tipo de etiqueta o genero por el cual se podrian recomendar por lo que me parecio correcto borrar

#### con esto eliminamos los registros con nulls en columnas importantes
    cabe aclarar que solo dropee donde tenian las 3 columnas nulas ya que se puede recomendar por cualquiera de las columnas
    title y app_name pueden usarse de igual maneraa para referir a el juego por lo que con que tengan una de ellas  2 se puede recomendar

## ahora asignaremos lo dtype correspondientes a cada campo

### "price" str a float

In [ ]:
lista = etf.valores_no_numericos(df_sg["price"])# esta funcion la hice en ETL_funciones
print(lista)

['Free To Play', 'Free to Play', 'None', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']


In [ ]:
#quito de la lista estos 2 valores que no se podian convertir a numerico pero contiene su precio numerico en la cadena
lista.remove("Starting at $499.00")
lista.remove('Starting at $449.00')

In [ ]:
#remplazo estas 2 cadenas por sus valores numericos
df_sg["price"] = df_sg["price"].replace("Starting at $499.00","499") 
df_sg["price"] = df_sg["price"].replace("Starting at $449.00","449") 

In [ ]:
lista #ahora la lista solo contiene valores que podria reemplazar por precio 0

['Free To Play',
 'Free to Play',
 'None',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Free to Try',
 'Free Movie',
 'Free to Use']

In [ ]:
df_sg["price"] = df_sg["price"].replace(lista,"0")#cambio los valores string que equivalen a precio 0 por "0"

In [ ]:
df_sg["price"] = df_sg["price"].astype(float)#cambiando el dtype a float

### "id" str a int

In [ ]:
lista = etf.valores_no_numericos(df_sg["id"])
lista #los valores en la columna "id" que no pueden ser convertidos a int

[]

In [ ]:
mask = (df_sg["id"] == "None")#filtro que selecciona solo las columnas con "id" con valor "None"
df_sg = df_sg[~mask]# ~ sirve para dropear , aca estamos dropeando las filas en que tienen id none

In [ ]:
df_sg["id"] = df_sg["id"].astype(int)

### "early-access"str a boolean

In [ ]:
df_sg["early_access"].value_counts()#como ya esta dicho en la parte donde se carga la libreria al cargar todo el df como str se cambiaron de sus valores float
# a sus correspondientes valores booleanos en formato str y al parecer no poseia datos nulos o incorrectos

early_access
False    30186
True      1947
Name: count, dtype: int64

In [ ]:
df_sg["early_access"] = df_sg["early_access"].astype(bool)# transformamos a boolean

#### tipos de datos(finales)

|columna|tipo|
|-------|----|
|publisher     | str|
|genres        | str(list)|
|app_name      | str|
|title         | str|
|url           | str|
|release_date  | str|
|tags          | str(list)|
|reviews_url   | str|
|specs         | str(list)|
|price         | float|
|early_access  | bool|
|id            | int|
|developer |str|

## guardando el df procesado en un archivo parquet

In [ ]:
df_sg.to_parquet(r"../app/processed_data/steam_games.parquet")#dentro de la carpeta de la api

<hr>

In [ ]:
reviews = []

with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", 'r', encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)


In [123]:
df_ur

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


## correccion los problemas en los datos encontrados durante la primer parte de el eda

## valores nulos

In [130]:
#verificando que reviews no este vacio ([])
df_ur[df_ur["reviews"] == "[]"] #al parecer no hay ninguna columna que sea literalmente una lista vacia por lo que por ahora no nos traera problemas

,user_id,user_url,reviews


## valores duplicados

In [131]:
df_ur.shape

(25799, 3)

In [133]:
df_ur["reviews"] = df_ur["reviews"].astype(str) #paso la columna a str para poder borrar los registros dupliados

In [134]:
df_ur.drop_duplicates(inplace=True)

In [135]:
df_ur.shape
#se borraron los 313 registros que identificamos en el eda

(25486, 3)

### tipos de datos(finales)
|columna |tipo|
|--------|----|
|user_id | str|
|user_url| str|
|reviews |str(list)|

## guardando el df procesado en un archivo parquet

In [136]:
df_ur.to_parquet(r"../app/processed_data/user_reviews.parquet")#dentro de la carpeta de la api

<hr>

In [125]:
items = []

with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))
df_ui = pd.DataFrame(items)

In [126]:
df_ui

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


## correccion los problemas en los datos encontrados durante la primer parte de el eda

### valores nulos

In [139]:
#tiene listas vacias en items


,user_id,items_count,steam_id,user_url,items


### valores duplicados

In [140]:
df_ui["items"] = df_ui["items"].astype(str)#convertirmos a str para poder verificar duplicados

In [141]:
df_ui.shape

(88310, 5)

In [143]:
df_ui.drop_duplicates(inplace=True)

In [144]:
df_ui.shape

(87653, 5)

## tipos de datos(finales)

|columna|tipo|
|-------|----|
|user_id|str|
|item_counts|int|
|steam_id|int|
|user_url|str|
|item|list|

## guardando el df procesado en un archivo parquet

In [ ]:
df_ui.to_parquet(r"../app/processed_data/users_items.parquet")#dentro de la carpeta de la api

<hr>